Task 1: File Management Script
   * Creates a directory named "backup" in the user's home directory.
   * Copies all .txt files from the current directory into the "backup"   directory.
   * Appends the current date and time to the filenames of the copied files.

Ans1.
  ```bash
    #!/bin/bash

    # Create a backup directory in the user's home directory
    backup_dir="$HOME/backup"
    mkdir -p "$backup_dir"

    # Loop through all .txt files in the current directory
    for file in *.txt; do
      if [[ -f "$file" ]]; then
          # Extract the filename without extension
          filename="${file%.txt}"
        
          # Append current date and time to the filename
          datetime=$(date +"%Y%m%d_%H%M%S")
          cp "$file" "$backup_dir/${filename}_$datetime.txt"
      fi
  done

  echo "Backup completed. Files are saved in: $backup_dir"

  ```

Task 2: System Health Check

Create a script thatg
- Checks the system’s CPU and memory usage^
- Reports if the CPU usage is above 80% or if the available memory is below 20%^
- Logs the results to a file named system_health.log.

Ans2.

  ```bash
    #!/bin/bash

    # Log file path
    log_file="system_health.log"

    # Get current date and time
    timestamp=$(date +"%Y-%m-%d %H:%M:%S")

    # Get CPU usage (ignoring the header and taking idle percentage)
    cpu_idle=$(top -bn1 | grep "Cpu(s)" | awk '{print $8}' | cut -d'.' -f1)
    cpu_usage=$((100 - cpu_idle))

    # Get memory usage percentage
    mem_total=$(free | grep Mem | awk '{print $2}')
    mem_available=$(free | grep Mem | awk '{print $7}')
    mem_usage=$(( ( (mem_total - mem_available) * 100 ) / mem_total ))
    mem_free_percent=$((100 - mem_usage))

    # Start log entry
    echo "[$timestamp] CPU Usage: ${cpu_usage}%, Free Memory: ${mem_free_percent}%" >> "$log_file"

    # Check CPU threshold
    if [ "$cpu_usage" -gt 80 ]; then
      echo "[$timestamp] WARNING: CPU usage above 80%!" >> "$log_file"
    fi

    # Check memory threshold
    if [ "$mem_free_percent" -lt 20 ]; then
      echo "[$timestamp] WARNING: Available memory below 20%!" >> "$log_file"
    fi

    echo "System health check completed and logged to $log_file"

  ```

Task 3: User Account Management

Write a script thatg
- Reads a list of usernames from a file (e.g., user_list.txt)^
- Creates a new user for each username^
- Generates a random password for each user and saves the username and password to a file named
credentials.txt.

Ans3.
  ```bash
    #!/bin/bash

    # Input file containing usernames
    user_file="user_list.txt"

    # Output file for storing generated credentials
    cred_file="credentials.txt"

    # Clear the credentials file before adding new entries
    > "$cred_file"

    # Loop through each username in the file
    while IFS= read -r username; do
      # Skip empty lines
      if [[ -z "$username" ]]; then
          continue
      fi

      # Create the user (without home directory to avoid clutter, you can remove -M if you want home dirs)
      sudo useradd -M "$username"

      # Generate a random password (12 characters)
      password=$(openssl rand -base64 12)

      # Set the password for the user
      echo "$username:$password" | sudo chpasswd

      # Save the username and password to credentials file
      echo "$username $password" >> "$cred_file"

    done < "$user_file"

    echo "User creation complete. Credentials saved in $cred_file"

  ```

Task 4: Automated Backup

Create a script thatg
- Takes a directory path as input from the user^
- Compresses the directory into a .tar.gz file^
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

Ans4.
  ```bash
    #!/bin/bash

    # Ask user for the directory path
    read -p "Enter the directory path to back up: " dir_path

    # Check if directory exists
    if [[ ! -d "$dir_path" ]]; then
      echo "Error: Directory does not exist."
      exit 1
    fi

    # Get the current date
    current_date=$(date +"%Y-%m-%d")

    # Create backup file name
    backup_file="backup_${current_date}.tar.gz"

    # Compress the directory
    tar -czf "$backup_file" -C "$(dirname "$dir_path")" "$(basename "$dir_path")"

    echo "Backup completed: $backup_file"

  ```

  * Run the Script:
      ```
        chmod +x automated_backup.sh
        ./automated_backup.sh

      ```

Task 5: Simple To-Do List

Create a Bash script thatg
- Implements a simple command-line to-do list^
- Allows the user to add tasks, view tasks, and remove tasks^
- Saves the tasks to a file (e.g., todo.txt)

Ans5:
  ```bash
#!/bin/bash

todo_file="todo.txt"

# Ensure the todo file exists
touch "$todo_file"

while true; do
    echo ""
    echo "===== Simple To-Do List ====="
    echo "1. View tasks"
    echo "2. Add a task"
    echo "3. Remove a task"
    echo "4. Exit"
    read -p "Choose an option (1-4): " choice

    case $choice in
        1)
            echo ""
            echo "===== Your Tasks ====="
            if [[ ! -s "$todo_file" ]]; then
                echo "No tasks found."
            else
                nl -w2 -s". " "$todo_file"
            fi
            ;;
        2)
            read -p "Enter your task: " task
            echo "$task" >> "$todo_file"
            echo "Task added."
            ;;
        3)
            if [[ ! -s "$todo_file" ]]; then
                echo "No tasks to remove."
            else
                nl -w2 -s". " "$todo_file"
                read -p "Enter the task number to remove: " num
                if [[ "$num" =~ ^[0-9]+$ ]]; then
                    sed -i "${num}d" "$todo_file"
                    echo "Task removed."
                else
                    echo "Invalid input."
                fi
            fi
            ;;
        4)
            echo "Goodbye!"
            break
            ;;
        *)
            echo "Invalid choice, please try again."
            ;;
    esac
done

  ```
  * Run the Script
    ```
      chmod +x todo.sh
      ./todo.sh

    ```

Task 6: Automated Software Installation

Write a script thatg
- Reads a list of software package names from a file (e.g., packages.txt).
- Installs each package using the appropriate package manager (apt, yum, etc.).
- Logs the installation status of each package.

Ans6:

  ```bash
#!/bin/bash

# File containing package names (one per line)
package_file="packages.txt"

# Log file to record installation status
log_file="install_log.txt"

# Clear previous log file
> "$log_file"

# Detect package manager
if command -v apt &> /dev/null; then
    pkg_manager="apt"
elif command -v yum &> /dev/null; then
    pkg_manager="yum"
elif command -v dnf &> /dev/null; then
    pkg_manager="dnf"
else
    echo "No supported package manager found (apt, yum, dnf)." | tee -a "$log_file"
    exit 1
fi

echo "Using package manager: $pkg_manager" | tee -a "$log_file"

# Update package list if using apt
if [ "$pkg_manager" == "apt" ]; then
    sudo apt update -y
fi

# Loop through packages
while IFS= read -r package; do
    # Skip empty lines or comments
    if [[ -z "$package" || "$package" =~ ^# ]]; then
        continue
    fi

    echo "Installing $package..." | tee -a "$log_file"

    if sudo $pkg_manager install -y "$package"; then
        echo "[SUCCESS] $package installed successfully." | tee -a "$log_file"
    else
        echo "[FAILED] Failed to install $package." | tee -a "$log_file"
    fi

done < "$package_file"

echo "Installation process completed. Log saved in $log_file"

  ```
  * Run the Script.
    ```
      chmod +x install_packages.sh
      sudo ./install_packages.sh

    ```
Task 7: Text File Processing

Create a script thatg
- Takes a text file as input.
- Counts and displays the number of lines, words, and characters in the file.
- Finds and displays the longest word in the file.

Ans7:
  ```bash
#!/bin/bash

# Ask for input file
read -p "Enter the text file path: " file

# Check if file exists
if [[ ! -f "$file" ]]; then
    echo "Error: File not found!"
    exit 1
fi

# Count lines, words, and characters
line_count=$(wc -l < "$file")
word_count=$(wc -w < "$file")
char_count=$(wc -m < "$file")

# Find the longest word
longest_word=$(tr -s '[:space:]' '\n' < "$file" | awk '{ if (length > max) { max = length; word = $0 } } END { print word }')

# Display results
echo "File: $file"
echo "Number of lines: $line_count"
echo "Number of words: $word_count"
echo "Number of characters: $char_count"
echo "Longest word: $longest_word"

  ```
  * Run the Script.
    ```
      chmod +x text_processing.sh
       ./text_processing.sh

    ```
